In [1]:
import requests
import os

In [2]:
def get_api_id():
    with open(".env", "r") as f:
        for line in f:
            listing = line.strip().split("=")
            if listing[0] == "DB_API_ID":
                return listing[1]

def get_api_secret():
    with open(".env", "r") as f:
        for line in f:
            listing = line.strip().split("=")
            if listing[0] == "DB_API_KEY":
                return listing[1]


In [3]:
def get_arrivals(eva_numbers, base_url):
    #url = "https://apis.deutschebahn.com/db-api-marketplace/apis/ris-stations/v1/connecting-times/REPLACE_EVANUMBER?group=REPLACE_THIS_VALUE&calculateSectors=REPLACE_THIS_VALUE&REPLACE_PERSONATYPES&REPLACE_SOURCE"
    headers = {
                "DB-Client-ID": get_api_id().replace("\'", ""),
                "DB-Api-Key": get_api_secret().replace("\'", ""),
                "accept": "application/vnd.de.db.ris+json"
            }
    arrivals_data = []

    for eva_number in eva_numbers:
        url = base_url + str(eva_number)
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            arrivals_data.append(response.json())
        else:
            print(f"Failed to fetch data for EVA number {eva_number}")

    return arrivals_data

In [4]:
url_connections="https://apis.deutschebahn.com/db-api-marketplace/apis/ris-stations/v1/connecting-times/"
url_platforms="https://apis.deutschebahn.com/db-api-marketplace/apis/ris-stations/v1/platforms/"
url_stations="https://apis.deutschebahn.com/db-api-marketplace/apis/ris-stations/v1/stations/" 
# stations API only have stationID
url_stops="https://apis.deutschebahn.com/db-api-marketplace/apis/ris-stations/v1/stop-places/"
# stops API have evanumber and stationID, so can be linked to that

In [29]:
# Example usage:
# connecting times are the time a traveler needs to change between platforms (evaNumbers)
eva_numbers = [8000105, 8000207, 8000304]  # Example EVA numbers
arrivals = get_arrivals(eva_numbers, url_connections)
for data in arrivals:
    for liste in data['connectingTimesList']:
        if(liste['fromEvaNumber'].__eq__(liste['toEvaNumber'])): continue
        else:
            print(liste['fromEvaNumber'], liste['toEvaNumber'], [(i['persona'],i['duration']) for i in liste['times'][:]])
             # Do whatever you need with the data

8000105 8089505 [('FREQUENT_TRAVELLER', 'PT15M'), ('HANDICAPPED', 'PT15M'), ('OCCASIONAL_TRAVELLER', 'PT15M')]
8000105 8089211 [('FREQUENT_TRAVELLER', 'PT15M'), ('HANDICAPPED', 'PT15M'), ('OCCASIONAL_TRAVELLER', 'PT15M')]
8000105 160865 [('FREQUENT_TRAVELLER', 'PT8M'), ('HANDICAPPED', 'PT8M'), ('OCCASIONAL_TRAVELLER', 'PT8M')]
8000105 100010 [('FREQUENT_TRAVELLER', 'PT10M'), ('HANDICAPPED', 'PT10M'), ('OCCASIONAL_TRAVELLER', 'PT10M')]
8000105 100008 [('FREQUENT_TRAVELLER', 'PT10M'), ('HANDICAPPED', 'PT10M'), ('OCCASIONAL_TRAVELLER', 'PT10M')]
8000105 8002041 [('FREQUENT_TRAVELLER', 'PT10M'), ('HANDICAPPED', 'PT10M'), ('OCCASIONAL_TRAVELLER', 'PT10M')]
8000105 8002042 [('FREQUENT_TRAVELLER', 'PT10M'), ('HANDICAPPED', 'PT10M'), ('OCCASIONAL_TRAVELLER', 'PT10M')]
8000105 8002050 [('FREQUENT_TRAVELLER', 'PT10M'), ('HANDICAPPED', 'PT10M'), ('OCCASIONAL_TRAVELLER', 'PT10M')]
8000105 102972 [('FREQUENT_TRAVELLER', 'PT10M'), ('HANDICAPPED', 'PT10M'), ('OCCASIONAL_TRAVELLER', 'PT10M')]
8000105 

In [30]:
# Example usage:
# stops are where trains arrive at a station (station plus platform)
eva_numbers = [8000105, 8000207, 8000304]  # Example EVA numbers
stops = get_arrivals(eva_numbers, url_stops)
for data in stops:
    print(data)  # Do whatever you need with the data

{'stopPlaces': [{'evaNumber': '8000105', 'stationID': '1866', 'names': {'DE': {'nameLong': 'Frankfurt(Main)Hbf', 'speechLong': 'Frankfurt am Main Hbf', 'speechShort': 'Frankfurt Hbf', 'synonyms': ['Francfort (Main)']}}, 'metropolis': {'DE': 'Frankfurt am Main'}, 'availableTransports': ['INTERCITY_TRAIN', 'CITY_TRAIN', 'INTER_REGIONAL_TRAIN', 'BUS', 'HIGH_SPEED_TRAIN', 'REGIONAL_TRAIN'], 'transportAssociations': ['RMV'], 'countryCode': 'DE', 'state': 'HE', 'municipalityKey': '06412000', 'timeZone': 'Europe/Berlin', 'position': {'longitude': 8.663789, 'latitude': 50.107145}}]}
{'stopPlaces': [{'evaNumber': '8000207', 'stationID': '3320', 'names': {'DE': {'nameLong': 'Köln Hbf', 'speechLong': 'Köln Hbf', 'speechShort': 'Köln Hbf', 'synonyms': ['Colonia(Köln)']}}, 'availableTransports': ['INTERCITY_TRAIN', 'CITY_TRAIN', 'INTER_REGIONAL_TRAIN', 'BUS', 'HIGH_SPEED_TRAIN', 'REGIONAL_TRAIN'], 'transportAssociations': ['VRS'], 'countryCode': 'DE', 'state': 'NW', 'municipalityKey': '05315000', '

In [7]:
stationIds=[]
for stopentry in stops:
    for stop in stopentry['stopPlaces']:
        stationIds.append(stop['stationID'])

In [37]:
# Example usage:
# stations are the single stations, referred to with stationID 
stations = get_arrivals(stationIds, url_stations)
for data in stations:
    
    print(data['stationID'], data['names']['DE']['name'], data['address']['country'], data['position']['longitude'], data['position']['latitude'])  # Do whatever you need with the data